# Introduction

## Présentation

Redis est une base de données NoSQL qui se classe dans la catégorie "data structure server". C'est  un système permettant de traiter, de manière très performante, des données sous la forme clés/valeurs. Cette performance est obtenue car les données sont stockées directement sur la mémoire RAM. Cela permet d'accéder très facilement et très rapidement aux données 

La clé permet de récupérer et d'identifier de manière unique la donnée. Redis ne permet pas de stocker des modèles de données très complexes, il faut donc ruser pour créer une structure de données complète et complexe.

Redis est souvent utilisé dans le developpement web pour gérer les sessions utilisateurs, leur panier, leurs configurations. En outre, lorsqu'on stocke une petite quantité de donnée et qu'on a besoin d'y accèder en quelques milliseconds

## Les données 



Les valeurs peuvent appartenir aux types suivants :

- chaînes de caractères
- compteurs numériques (les données numériques seront stockées dans la base sous la forme de chaîne de caractères)
- listes 
- sets ou sets ordonnés
- dictionnaires

## Lancer un container Redis

Comme pour Mongo, il nous faut une instance redis qui tourne en local, pour se faire il suffit de lancer un container Redis

Vous devriez être familier avec Docker maintenant, à vous de jouer pour lancer ce container !

## Installation

Le package Redis pour python est déjà installé, voici la commande si jamais :

`pipenv install redis==5.0.8`

# Let's play

In [3]:
import redis

In [4]:
redis_client = redis.StrictRedis(host='localhost')

Pour tester la connexion on peut pinguer la base de données :

In [5]:
redis_client.ping()

True

Vous pouvez regarder toutes les configurations utilisées par Redis, beaucoup ne sont pas utiles à notre niveau, mais vous pouvez voir par exemple, `database`, `maxclients`, `port` , `timeout` qui peuvent être utile de modifier pour différentes applications

In [6]:
redis_client.config_get()

{'set-proc-title': 'yes',
 'replica-read-only': 'yes',
 'lazyfree-lazy-user-flush': 'no',
 'repl-diskless-sync': 'yes',
 'unixsocket': '',
 'rdbcompression': 'yes',
 'set-max-listpack-entries': '128',
 'repl-ping-replica-period': '10',
 'cluster-announce-bus-port': '0',
 'maxmemory-eviction-tenacity': '10',
 'appendonly': 'no',
 'io-threads': '1',
 'slave-announce-ip': '',
 'max-new-connections-per-cycle': '10',
 'proc-title-template': '{title} {listen-addr} {server-mode}',
 'syslog-facility': 'local0',
 'repl-timeout': '60',
 'supervised': 'no',
 'cluster-node-timeout': '15000',
 'activedefrag': 'no',
 'lua-time-limit': '5000',
 'cluster-replica-no-failover': 'no',
 'tcp-keepalive': '300',
 'save': '3600 1 300 100 60 10000',
 'protected-mode': 'no',
 'lfu-log-factor': '10',
 'cluster-require-full-coverage': 'yes',
 'enable-protected-configs': 'no',
 'server-cpulist': '',
 'oom-score-adj-values': '0 200 800',
 'tls-key-file': '',
 'io-threads-do-reads': 'no',
 'cluster-replica-validity

## Stockage clé valeurs

Pour stocker une valeur correspondant à une clé 

In [7]:
KEY = "name"
VALUE = "yourname"
redis_client.set(KEY, VALUE)

True

True montre que le stockage s'est bien déroulé. 

Pour récupérer la valeur stockée il suffit d'appeler la méthode `GET` sur cette clé

In [8]:
redis_client.get(KEY)

b'yourname'

Vous pouvez voir qu'à l'aide du `b` que la réponse n'est pas exactement ce que l'on a stocké, mais une représentation en bytes de notre chaine de charactères. 

In [9]:
assert redis_client.get(KEY) == VALUE, "Les valeurs ne correspondent pas"

AssertionError: Les valeurs ne correspondent pas

Pour cela, en python, il faut appeler la méthode `decode()` pour transformer des bytes en string 

In [ ]:
assert redis_client.get(KEY).decode() == VALUE, "les valeurs ne correspondent pas"

On peut préciser au client de décoder les réponses et de les transformer directement en strings

In [ ]:
redis_client = redis.StrictRedis(host='localhost', charset='utf-8', decode_responses=True)

In [ ]:
assert redis_client.get(KEY) == VALUE

Les valeurs peuvent aussi être des entiers

In [ ]:
KEY = "age"
VALUE = 20

redis_client.set(KEY, VALUE)

In [ ]:
redis_client.get(KEY)

Vous pouvez alors utiliser la méthode `incr` pour incrémenter cet entier. Il faut faire attention puisque redis renvoie toujours les entiers comme des chaines de charactères.

In [ ]:
redis_client.incr(KEY)

## Exemple 

In [10]:
from ipywidgets import widgets, interact
from IPython.display import display

In [ ]:
KEY = "click_number"
DEFAULT_VALUE = 0

redis_client.set(KEY, DEFAULT_VALUE)

button = widgets.Button(description="Click me!")
def on_button_clicked(button):
	redis_client.incr(KEY)
	print(f"Le bouton à été cliqué {redis_client.get(KEY)} fois")
	button.description = "Click me more !"
	
button.on_click(on_button_clicked)
display(button)

Button(description='Click me!', style=ButtonStyle())

Le bouton à été cliqué b'1' fois
Le bouton à été cliqué b'2' fois
Le bouton à été cliqué b'3' fois
Le bouton à été cliqué b'4' fois
Le bouton à été cliqué b'5' fois
Le bouton à été cliqué b'6' fois


In [12]:
print(f"Tu as cliqué sur le bouton {redis_client.get(KEY)} fois !")

Tu as cliqué sur le bouton b'6' fois !


## Les Listes

Redis permet aussi de stocker des valeurs de type `list`, pour cela nous avons à notre disposition un ensemble de méthodes :  

- La méthode `LPUSH`  permet d'ajouter un élément en tête de list la complexité est de O(1).  
- La méthode `RPUSH` permet d'ajouter un élément en queue de list la complexité est de O(1).  
- La méthode `LINSERT` permet d'ajouter un élément avant ou après la première occurence d'une certaine valeur, la complexité est de O(N) à part pour le premier et le dernier élèment.
- La méthode `LSET` permet de modifier la valeur correspondant à l'index spécifié, la complexité est de O(N) à part pour le premier et le dernier élèment.
- La méthode `RPOP` permet de supprimer un élément en queue de list la complexité est de O(1).
- La méthode `LPOP` permet de supprimer un élément en tête de list la complexité est de O(1).
- La méthode `LINDEX` permet d'afficher un élément en fonction de son index.
- La méthode `LREM` permet de supprimer un élément en fonction de son index.
- La méthode `LLEN` permet de récupérer la taille d'une liste.

Nous allons toutes les tester.

In [13]:
redis_client.lpush('language_list', "kotlin")
redis_client.lpush('language_list', "python")

2

On peut aussi ajouter plusieurs éléments en même temps dans une liste

In [14]:
redis_client.lpush('language_list', "java", "c++")

4

In [15]:
f"La taille de la liste est {redis_client.llen('language_list')}"

'La taille de la liste est 4'

In [16]:
INDEX = 2
f"L'élément à l'index {INDEX} de la liste est '{redis_client.lindex('language_list', INDEX)}'"

"L'élément à l'index 2 de la liste est 'b'python''"

Le client renvoie la taille de la liste venant d'être mise à jour. On peut afficher tous les éléments de la liste en les retirant un par un.

In [ ]:
while(redis_client.llen('language_list')!=0):

	print(redis_client.lpop('language_list'))

## Les dictionnaires 

Redis permet de stocker assez facilement des dictionnaires, on peut comme cela stocker des structures de données plus complexes. Pour cela, nous avons plusieurs méthodes : 

- La méthode `HMSET` permet d'ajouter un dictionnaire à une clé
- La méthode `HSET` permet d'ajouter ou de modifier la valeur d'une clé d'un dictionnaire
- La méthode `HGET` permet d'ajouter ou de modifier la valeur d'une clé d'un dictionnaire
- La méthode `HGETALL` permet de récupérer toutes les clés valeurs d'un dictionnaire.
- La méthode `HKEYS` permet de récupérer toutes les clés d'un dictionnaire
- La méthode `HVALS` permet de récupérer toutes les valeurs d'un dictionnaire

In [ ]:
doc = {
	"name":"yourname",
	"age":20,
	"city":"paris"
}

In [ ]:
redis_client.hset("user1", mapping=doc)

[b'yourname', b'20', b'paris']

In [92]:
print(redis_client.hgetall("user1"))
print(redis_client.hkeys("user1"))
print(redis_client.hvals("user1"))

{'name': 'yourname', 'age': '20', 'city': 'paris'}
['name', 'age', 'city']
['yourname', '20', 'paris']


On peut modifier uniquement une clé du dictionnaire

In [22]:
redis_client.hset("user1", "city", "lyon")

0

In [23]:
redis_client.hgetall("user1")

{b'name': b'yourname', b'age': b'20', b'city': b'lyon'}

## Les pipelines

In [25]:
import timeit

Lorsque l'on appelle une méthode depuis le client python, chaqu'une d'entre elles fait un appel à Redis, ce qui peut prendre du temps. Les pipelines permettent de réduire ce temps en stockant toutes les opérations et en les applicant une seule fois sur la base. Cela permet d'augmenter grandement les performances en réduisant les appels et le passage par le réseau. 

In [26]:
N = 10000

In [ ]:
def store_key_values(N):
	for i in range(N):
		redis_client.set(f"key_{i}", f"value_{i}")

In [28]:
%%timeit
store_key_values(N=N)

13.9 s ± 497 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
def store_key_values_with_pipelines(N):
	p = redis_client.pipeline()
	for i in range(N):
		p.set(f"key_{i}", f"value_{i}")

In [30]:
%%timeit
store_key_values_with_pipelines(N=N)

19.3 ms ± 1.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


On voit alors que l'on peut diviser le temps par 100. Dans vos différents métiers les performances sont souvent très importantes

# Exercice

Utiliser redis pour gérer les commandes d'un restaurant de burgers, pour simplifier les choses les clients pourront choisir leurs ingrédients dans 3 types de produits :

- Pain: blanc, complet, aux céréales
- Garniture : salade, tomates , oignons, concombres
- Protéines : poulet, boeuf, agneau, tofu

Chaque ingrédient équivaut à un prix et un poids, avec chaque commande, vous devrez calculer le poids et le prix que les clients devront régler. 

1) Stocker les informations concernant les ingrédients dans redis   
2) Créer la méthode permettant de créer aléatoirement un burger  
3) Créer la méthode qui permet d'empiler dans redis les prochaines commandes (Réfléchir au type de données utilisé)  
4) Créer la méthode permettant de calculer le poids et le prix du burger  
5) Créer une boucle pour dépiler, réaliser et servir les burgers.  

Pour vous aider voici quelques données 

In [ ]:
pains = {
	"blanc":{
		"poid":100,
		"prix": 2
	},
	"complet":{
		"poid":90,
		"prix": 3
	},
	"cereales":{
		"poid":120,
		"prix": 4
	}
}

In [ ]:
garniture = {
	"salade":{
		"poid":50,
		"prix": 1
	}, "tomates":{
		"poid":60,
		"prix": 2
	} , "oignons":{
		"poid":10,
		"prix": 1.5
	}, "concombres":{
		"poid":30,
		"prix": 3
	}
}

In [ ]:
proteine = {
	"poulet":{
		"poid":100,
		"prix": 6
	}, "boeuf":{
		"poid":150,
		"prix": 5
	} , "agneau":{
		"poid":120,
		"prix": 7
	}, "tofu":{
		"poid":80,
		"prix": 8
	}
}

In [117]:
import random

In [216]:

# 1) Stocker les informations concernant les ingrédients dans Redis
def store_ingredients():
    # Stockage des pains dans Redis
    for pain, details in pains.items():
        redis_client.hset(f"pain:{pain}", mapping={"poid": details["poid"], "prix": details["prix"]})


    # Stockage des garnitures dans Redis
    for garnitures, details in garniture.items():
        redis_client.hset(f"garnitures:{garnitures}", mapping={"poid": details["poid"], "prix": details["prix"]})


    # Stockage des protéines dans Redis
    for proteine_name, details in proteine.items():
        redis_client.hset(f"proteine_name:{proteine_name}", mapping={"poid": details["poid"], "prix": details["prix"]})

# 2) Créer la méthode permettant de créer aléatoirement un burger
def make_burger():
    burger_id = redis_client.incr("burger_id_counter")
    burger_key = f"burger:{burger_id}"
    burger = {
        "pain": random.choice(list(pains.keys())),
        "garnitures": ",".join(random.sample(list(garniture.keys()), random.randint(1, len(garniture)))),
        "proteine": random.choice(list(proteine.keys()))
    }
    redis_client.hset(burger_key, mapping=burger)
    return burger_key

# 3) Créer la méthode qui permet d'empiler dans Redis les prochaines commandes
def make_order():
    burger_key = make_burger()
    redis_client.lpush("order", burger_key)

# 4) Créer la méthode permettant de calculer le poids et le prix du burger
def process_price_weight(burger_key):
    price, weight = 0, 0
    burger = redis_client.hgetall(burger_key)
    
    # Calcul du prix et du poids du pain
    pain = burger["pain"]
    price += float(redis_client.hget(f"pain:{pain}", "prix"))
    weight += float(redis_client.hget(f"pain:{pain}", "poid"))
    
    # Calcul du prix et du poids des garnitures
    garnitures = burger["garnitures"].split(",")
    for garnitures in garnitures:
        price += float(redis_client.hget(f"garniture:{garnitures}", "prix"))
        weight += float(redis_client.hget(f"garniture:{garnitures}", "poid"))
    
    # Calcul du prix et du poids de la protéine
    proteine_name = burger["proteine"]
    price += float(redis_client.hget(f"proteine:{proteine_name}", "prix"))
    weight += float(redis_client.hget(f"proteine:{proteine_name}", "poid"))
    
    return price, weight

# 5) Créer une boucle pour dépiler, réaliser et servir les burgers
def process_burger():
    burger_key = redis_client.rpop("order")
    if burger_key:
        price, weight = process_price_weight(burger_key)
        return burger_key, price, weight
    return None, 0, 0

# Exemple d'utilisation
store_ingredients()  # Stocker les ingrédients dans Redis

# Créer des commandes aléatoires
for _ in range(10):
    make_order()



In [217]:
while True:
	burger, price, weight = process_burger()
	print(f'Le burger {burger} de {weight} gr à été préparé et servi il faut régler {price}€')
	if burger is None:
		break
	

Le burger burger:36 de 320.0 gr à été préparé et servi il faut régler 10.0€
Le burger burger:37 de 340.0 gr à été préparé et servi il faut régler 14.5€
Le burger burger:38 de 290.0 gr à été préparé et servi il faut régler 15.5€
Le burger burger:39 de 340.0 gr à été préparé et servi il faut régler 16.5€
Le burger burger:40 de 350.0 gr à été préparé et servi il faut régler 15.5€
Le burger burger:41 de 330.0 gr à été préparé et servi il faut régler 16.0€
Le burger burger:42 de 350.0 gr à été préparé et servi il faut régler 15.5€
Le burger burger:43 de 300.0 gr à été préparé et servi il faut régler 13.0€
Le burger burger:44 de 420.0 gr à été préparé et servi il faut régler 16.5€
Le burger burger:45 de 290.0 gr à été préparé et servi il faut régler 15.5€
Le burger None de 0 gr à été préparé et servi il faut régler 0€


'burger:15'